# Workforce Scheduling Problem

# Description:
A company operates 7 days a week and needs to assign employees to daily shifts.
The company must meet the minimum daily staffing requirements, and each employee
is allowed to work a maximum number of days per week to avoid burnout. Employees
have different hourly wages, and the shifts are 8 hours long. The objective is
to schedule employees in a way that satisfies the following constraints:

1. The minimum number of employees required for each day is met.
2. No employee works more than their allowed maximum workdays in the week.
3. The total cost of wages is minimized.

Definitions:
- Number of employees: 5
- Number of days: 7
- Hourly wages for employees: [15, 20, 18, 22, 19] (in Euro per hour)
- Maximum workdays per employee: [5, 5, 4, 4, 5]
- Daily staffing requirements: [3, 2, 4, 3, 2, 3, 4] (number of employees required each day)
- Shift duration: 8 hours

Goal:
To determine the optimal schedule that minimizes the total wage cost while adhering
to the constraints.

Output:
- The optimal total cost of scheduling.
- The daily work schedule for each employee.

In [ ]:
!pip install Pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 79.0 MB/s eta 0:00:00


In [20]:
from pulp import *

In [21]:
# Define the data
num_employees = 5
num_days = 7

# Hourly wages for each employee
wages = [15, 20, 18, 22, 19]

# Maximum workdays for each employee
max_workdays = [5, 5, 4, 4, 5]

# Daily staffing requirements (minimum employees required each day)
daily_requirements = [3, 2, 4, 3, 2, 3, 4]

# Shift duration in hours
shift_hours = 8


In [22]:
# Create the linear programming problem (maximize)
problem = LpProblem("Workforce_Scheduling", LpMinimize)

In [23]:
# x[i][j] is a binary variable indicating if employee i is assigned to day j
x = [
    [LpVariable(f"x_{i}_{j}", cat="Binary") for j in range(num_days)]
    for i in range(num_employees)
]

In [24]:
# Objective function: Minimize total wage cost
problem += lpSum(
    wages[i] * shift_hours * x[i][j] for i in range(num_employees) for j in range(num_days)
), "Total_Wage_Cost"

In [25]:
# Constraints: Ensure daily staffing requirements are met
for j in range(num_days):
    problem += (
        lpSum(x[i][j] for i in range(num_employees)) >= daily_requirements[j],
        f"Daily_Requirement_Day_{j+1}",
    )

In [26]:
# Constraints: Ensure no employee works more than their allowed maximum workdays
for i in range(num_employees):
    problem += (
        lpSum(x[i][j] for j in range(num_days)) <= max_workdays[i],
        f"Max_Workdays_Employee_{i+1}",
    )


In [27]:
# Solve the problem
problem.solve()

1

In [28]:
# Print the results
print(f"Status: {LpStatus[problem.status]}")
if LpStatus[problem.status] == "Optimal":
    print(f"Total Cost: ${value(problem.objective):.2f}\n")
    print("Work Schedule:")
    for i in range(num_employees):
        schedule = [f"Day {j+1}" for j in range(num_days) if x[i][j].varValue == 1]
        print(f"  Employee {i+1}: {', '.join(schedule) if schedule else 'No shifts assigned'}")
else:
    print("No optimal solution found.")

Status: Optimal
Total Cost: $3088.00

Work Schedule:
  Employee 1: Day 2, Day 4, Day 5, Day 6, Day 7
  Employee 2: Day 1, Day 3, Day 4, Day 6, Day 7
  Employee 3: Day 1, Day 3, Day 5, Day 7
  Employee 4: Day 1, Day 3
  Employee 5: Day 2, Day 3, Day 4, Day 6, Day 7
